This notebook replicates key steps of "The Macro-Dynamics of Sorting between Workers and Firms" by Jeremy Lise and Jean-Marc Robin: https://www.aeaweb.org/articles?id=10.1257/aer.20131118

# Model
The economy consists of workers with ability $x \sim Beta(\beta_1,\beta_2)$ and firms with productivity $y\sim{U}(y)$, both with a measure of one. The beginning of period mass of employed and unemployed workers, $h_t(x,y)$ and $u_t(x)$, are such that $u_t(x) + \int h_t(x,y)\,dy=l(x)$. 

An aggregate shock, $z$, which follows a Markov chain with transition probability $\pi(z,z')$, hits the economy and might result in the termination of a job. The post-shock measure are $h_{t+}(x,y)$ and $u_{t+}(x)$. 

It is assumed that workers costlessly search with different intensities: employed workers search with $s$ and unemloyed with $1$. The aggregate effort is $L_t = \int 1\cdot u_{t+}(x)\, dx + \int \int s \cdot h_{t+}(x,y) \, dx \,dy$. Firms post $v_t(y)$ job opportunities for a cost $c(v)$. Aggregate opportunities are $V_t = \int v_t(y)\,dy$. Meetings are assigned randomly with meeting technology $M_t=M(L_t, V_t)=\min \Big( \alpha L_t^\omega V_t^{1-\omega}, L_t, V_t \Big)$. The meeting probabilities for an unemployed and an employed worker are $\lambda_t=\frac{M_t}{L_t}$ and $s\lambda_t$. The meeting probabiliby per posted opportunity is $q_t=\frac{M_t}{V_t}$.

The joint value of a match, $P_t(x,y)$, is the value of a match and includes continuation values of the worker and the firm. The value of being unemployed is $B_{t}(x)$. A match is dissolved if $\mathbb{1}\Big(P_{t+1}(x,y)<B_{t+1}(x)\Big) + \delta \cdot \mathbb{1}\Big(P_{t+1}(x,y) \geq B_{t+1}(x)\Big)$, where $\delta$ is the exogenous seperation rate.

The unemployed worker engages in home production $b(x,z_t)$ and values being unemployed:
$$B_t(x) = b(x,z_t) + \frac{1}{1+r} \mathbb{E}_t B_{t+1}(x)$$
Let $S_t(x,y) = P_t(x,y) - B_t(x)$ be the match surplus, then proposition 1 defines the equilibrium to be such that:

$$S(x,y,z) = s(x,y,z) + \frac{1-\delta}{1+r} \int S(x,y,z')^+ \pi(z,z') \, dz'$$
where $s(x,y,z) = p(x,y,z) - b(x,z)$ and $x^+ = \max\Big(x,0\Big)$.



# Implementation

In [8]:
using QuantEcon

In [9]:
?tauchen()

search: tauchen



Tauchen's (1996) method for approximating AR(1) process with finite markov chain

The process follows

$$
    y_t = \mu + \rho y_{t-1} + \epsilon_t
$$

where $\epsilon_t \sim N (0, \sigma^2)$

##### Arguments

  * `N::Integer`: Number of points in markov process
  * `ρ::Real` : Persistence parameter in AR(1) process
  * `σ::Real` : Standard deviation of random component of AR(1) process
  * `μ::Real(0.0)` : Mean of AR(1) process
  * `n_std::Integer(3)` : The number of standard deviations to each side the process should span

##### Returns

  * `mc::MarkovChain{Float64}` : Markov chain holding the state values and transition matrix


In [ ]:
struct Model
    #deep parameters
    β1::Float64
    β2::Float64
    c0::Float64
    c1::Float64
    α::Float64
    ω::Float64
    r::Float64

    p1::Float64
    p2::Float64
    p3::Float64
    p4::Float64
    p5::Float64
    p6::Float64

    σ::Float64
    ρ::Float64
    s::Float64

    #technical
    dt::Float64
    ϵ::Float64

    Nx::Int64
    Ny::Int64
    Nz::Int64
    Gridx::Vector{Float64}
    Gridy::Vector{Float64}
    Gridz::Vector{Float64}

    Π::Matrix{Float64}
end
#β1, β2, c0, c1, α, ω, r, p1, p2, p3, p4, p5, p6, σ, ρ, s, dt, ϵ, Nx, Ny, Nz, Gridx, Gridy, Gridz

function Model(;β1=2.148, β2=12.001, c0=0.028, c1=0.084, α=0.497, ω=1/2, r=0.05, p1=0.003, p2=2.053, p3=-0.140, p4=8.035, p5=-1.907, p6=6.596, σ=0.071, ρ=0.999, s=0.027, dt=1/52, ϵ=0.001, Nx=21, Ny=21, Nz=51)
    
    Gridx = collect(linspace(ϵ, 1-ϵ, Nx))
    Gridy = collect(linspace(ϵ, 1-ϵ, Ny))
    mc = tauchen(Nz, ρ, σ * √(1-ρ^2))
    Π = 

    
    return Model(β1, β2, c0, c1, α, ω, r, p1, p2, p3, p4, p5, p6, σ, ρ, s, dt, ϵ, Nx, Ny, Nz,
        Gridx, Gridy, Gridz, Π)

end

In [11]:
mc = tauchen(51, 0.999, 0.071 * √(1-0.999^2))

Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
[0.898824 0.101139 … 0.0 0.0; 0.079804 0.819494 … 0.0 0.0; … ; 0.0 0.0 … 0.819494 0.079804; 0.0 0.0 … 0.101139 0.898824]

In [12]:
mc.state_values

51-element Array{Float64,1}:
 -0.213  
 -0.20448
 -0.19596
 -0.18744
 -0.17892
 -0.1704 
 -0.16188
 -0.15336
 -0.14484
 -0.13632
 -0.1278 
 -0.11928
 -0.11076
  ⋮      
  0.11928
  0.1278 
  0.13632
  0.14484
  0.15336
  0.16188
  0.1704 
  0.17892
  0.18744
  0.19596
  0.20448
  0.213  

In [7]:
linspace(0,1,10)

0.0:0.1111111111111111:1.0

In [6]:
model = 

LoadError: [91mMethodError: no method matching Model()[0m
Closest candidates are:
  Model([91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Float64[39m, [91m::Int64[39m, [91m::Int64[39m, [91m::Int64[39m, [91m::Array{Float64,1}[39m, [91m::Array{Float64,1}[39m, [91m::Array{Float64,1}[39m) at In[5]:3
  Model([91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::